이 실습은 Runpod에서 A100-SXM GPU 1개, container disk 50GB (이상)에서 진행되었습니다.

In [ ]:
%pip install "torch==2.4.0" "torchvision==0.19.0" "torchaudio==2.4.0"
%pip install "transformers==4.45.1" "datasets==3.0.1" "accelerate==0.34.2" "trl==0.11.1" "peft==0.13.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 21.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 63.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 137.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 123.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 18.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 54.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 105.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 58.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 41.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 49.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━

## 1. 데이터 전처리

In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [2]:
# 1. 허깅페이스 허브에서 데이터셋 로드
dataset = load_dataset("iamjoon/ecommerce-function-calling-datasets-korean", split="train")

In [3]:
# 테스트 비율 설정
test_ratio = 0.2

# 전체 길이와 테스트 데이터 크기 계산
total_len = len(dataset)
test_size = int(total_len * test_ratio)

# 앞에서부터 테스트 데이터, 나머지는 학습 데이터
test_indices = list(range(test_size))
train_indices = list(range(test_size, total_len))

In [4]:
# OpenAI 포맷으로 변환 함수
def format_conversations(sample):
    return {
        "messages": [
            {"role": "system", "content": sample["system_prompt"]},
            *sample["messages"]
        ]
    }

# 분할 및 변환
train_dataset = [format_conversations(dataset[i]) for i in train_indices]
test_dataset = [format_conversations(dataset[i]) for i in test_indices]

# 리스트를 다시 HuggingFace Dataset 객체로 변환
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)

# 결과 확인
print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")


전체 데이터 분할 결과: Train 311개, Test 77개


In [5]:
train_dataset[145]["messages"]

[{'content': '당신은 상준몰의 AI 상담사입니다. 성심성의껏 상담하십시오.\n\n로그인한 사용자의 현재 ID: U010\n오늘 날짜: 2024-07-24\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "view_order_history", "description": "사용자의 전체 주문 내역을 반환합니다. 각 주문에 대해 주문 번호, 주문 일자, 총 결제 금액, 결제 상태, 배송 상태, 택배사, 운송장 번호, 배송 진행 단계, 주문에 포함된 상품명 목록을 제공합니다. 이 함수로도 최근 주문의 배송 상태를 확인할 수 있습니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "description": "주문 내역을 조회할 사용자의 고유 식별자 (예: \'U001\')"}}, "required": ["user_id"], "additionalProperties": false}}}\n{"type": "function", "function": {"name": "show_cart", "description": "지정된 사용자(user_id)의 현재 장바구니에 담긴 모든 상품을 반환합니다. 각 항목에 대해 상품 ID, 상품명, 가격, 수량, 장바구니에 추가된 일시를 포함합니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "description": "장바구니를 조회할 사용자의 고유 식별자 (예: \'U001\')"}}, "required": ["us

## 2. 모델 로드 및 템플릿 적용

In [8]:
# 허깅페이스 모델 ID
model_id = "Qwen/Qwen2.5-7B-Instruct"

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [26]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[145]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 상준몰의 AI 상담사입니다. 성심성의껏 상담하십시오.

로그인한 사용자의 현재 ID: U010
오늘 날짜: 2024-07-24

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "view_order_history", "description": "사용자의 전체 주문 내역을 반환합니다. 각 주문에 대해 주문 번호, 주문 일자, 총 결제 금액, 결제 상태, 배송 상태, 택배사, 운송장 번호, 배송 진행 단계, 주문에 포함된 상품명 목록을 제공합니다. 이 함수로도 최근 주문의 배송 상태를 확인할 수 있습니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "description": "주문 내역을 조회할 사용자의 고유 식별자 (예: 'U001')"}}, "required": ["user_id"], "additionalProperties": false}}}
{"type": "function", "function": {"name": "show_cart", "description": "지정된 사용자(user_id)의 현재 장바구니에 담긴 모든 상품을 반환합니다. 각 항목에 대해 상품 ID, 상품명, 가격, 수량, 장바구니에 추가된 일시를 포함합니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "description": "장바구니를 조회할 사용자의 고유 식별자 (예: 'U001')"}}, "required": ["user_id"], "a

## 3. LoRA와 SFTConfig 설정

In [27]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

`lora_alpha`: LoRA(Low-Rank Adaptation)에서 사용하는 스케일링 계수를 설정합니다. LoRA의 가중치 업데이트가 모델에 미치는 영향을 조정하는 역할을 하며, 일반적으로 학습 안정성과 관련이 있습니다.

`lora_dropout`: LoRA 적용 시 드롭아웃 확률을 설정합니다. 드롭아웃은 과적합(overfitting)을 방지하기 위해 일부 뉴런을 랜덤하게 비활성화하는 정규화 기법입니다. 0.1로 설정하면 학습 중 10%의 뉴런이 비활성화됩니다.

`r`: LoRA의 랭크(rank)를 설정합니다. 이는 LoRA가 학습할 저차원 공간의 크기를 결정합니다. 작은 값일수록 계산 및 메모리 효율이 높아지지만 모델의 학습 능력이 제한될 수 있습니다.

`bias`: LoRA 적용 시 편향(bias) 처리 방식을 지정합니다. "none"으로 설정하면 편향이 LoRA에 의해 조정되지 않습니다. "all" 또는 "lora_only"와 같은 값으로 변경하여 편향을 조정할 수도 있습니다.

`target_modules`: LoRA를 적용할 특정 모듈(레이어)의 이름을 리스트로 지정합니다. 예제에서는 "q_proj"와 "v_proj"를 지정하여, 주로 Self-Attention 메커니즘의 쿼리와 값 프로젝션 부분에 LoRA를 적용합니다.

`task_type:` LoRA가 적용되는 작업 유형을 지정합니다. "CAUSAL_LM"은 Causal Language Modeling, 즉 시퀀스 생성 작업에 해당합니다. 다른 예로는 "SEQ2SEQ_LM"(시퀀스-투-시퀀스 언어 모델링) 등이 있습니다.

In [28]:
args = SFTConfig(
    output_dir="qwen-2.5-7b-function-calling",           # 저장될 디렉토리와 저장소 ID
    num_train_epochs=3,                      # 학습할 총 에포크 수
    per_device_train_batch_size=2,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None
)

`output_dir`: 학습 결과가 저장될 디렉토리 또는 모델 저장소의 이름을 지정합니다. 이 디렉토리에 학습된 모델 가중치, 설정 파일, 로그 파일 등이 저장됩니다.

`num_train_epochs`: 모델을 학습시키는 총 에포크(epoch) 수를 지정합니다. 에포크는 학습 데이터 전체를 한 번 순회한 주기를 의미합니다. 예를 들어, `3`으로 설정하면 데이터셋을 3번 학습합니다.

`per_device_train_batch_size`: GPU 한 대당 사용되는 배치(batch)의 크기를 설정합니다. 배치 크기는 모델이 한 번에 처리하는 데이터 샘플의 수를 의미합니다. 작은 크기는 메모리 사용량이 적지만 학습 시간이 증가할 수 있습니다.

`gradient_accumulation_steps`: 그래디언트를 누적할 스텝(step) 수를 지정합니다. 이 값이 2로 설정된 경우, 두 스텝마다 그래디언트를 업데이트합니다. 배치 크기를 가상으로 늘리는 효과가 있으며, GPU 메모리 부족 문제를 해결할 때 유용합니다.

`gradient_checkpointing`: 그래디언트 체크포인팅을 활성화하여 메모리를 절약합니다. 이 옵션은 계산 그래프를 일부 저장하지 않고 다시 계산하여 메모리를 절약하지만, 속도가 약간 느려질 수 있습니다.

`optim`: 학습 시 사용할 최적화 알고리즘을 설정합니다. `adamw_torch_fused`는 PyTorch의 효율적인 AdamW 최적화기를 사용합니다.

`logging_steps`: 로그를 기록하는 주기를 스텝 단위로 지정합니다. 예를 들어, `10`으로 설정하면 매 10 스텝마다 로그를 기록합니다.

`save_strategy`: 모델을 저장하는 전략을 설정합니다. `"steps"`로 설정된 경우, 지정된 스텝마다 모델이 저장됩니다.

`save_steps`: 모델을 저장하는 주기를 스텝 단위로 설정합니다. 예를 들어, `50`으로 설정하면 매 50 스텝마다 모델을 저장합니다.

`bf16`: bfloat16 정밀도를 사용하도록 설정합니다. bfloat16은 FP32와 유사한 범위를 제공하면서 메모리와 계산 효율성을 높입니다.

`learning_rate`: 학습률을 지정합니다. 학습률은 모델의 가중치가 한 번의 업데이트에서 얼마나 크게 변할지를 결정합니다. 일반적으로 작은 값을 사용하여 안정적인 학습을 유도합니다.

`max_grad_norm`: 그래디언트 클리핑의 임계값을 설정합니다. 이 값보다 큰 그래디언트가 발생하면, 임계값으로 조정하여 폭발적 그래디언트를 방지합니다.

`warmup_ratio`: 학습 초기 단계에서 학습률을 선형으로 증가시키는 워밍업 비율을 지정합니다. 학습의 안정성을 높이기 위해 사용됩니다.

`lr_scheduler_type`: 학습률 스케줄러의 유형을 설정합니다. `"constant"`는 학습률을 일정하게 유지합니다.

`push_to_hub`: 학습된 모델을 허브에 업로드할지 여부를 설정합니다. `False`로 설정하면 업로드하지 않습니다.

`remove_unused_columns`: 사용되지 않는 열을 제거할지 여부를 설정합니다. True로 설정하면 메모리를 절약할 수 있습니다.

`dataset_kwargs`: 데이터셋 로딩 시 추가적인 설정을 전달합니다. 예제에서는 `skip_prepare_dataset: True`로 설정하여 데이터셋 준비 단계를 건너뜁니다.

`report_to`: 학습 로그를 보고할 대상을 지정합니다. `None`으로 설정되면 로그가 기록되지 않습니다.

## 4. 학습 중 전처리 함수: collate_fn


In [29]:
import torch

def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    # Qwen ChatML 토큰 정의
    start_token = "<|im_start|>"
    end_token   = "<|im_end|>"

    # assistant prefix와 토큰 시퀀스
    assistant_prefix = f"{start_token}assistant\n"
    assistant_tokens = tokenizer.encode(assistant_prefix, add_special_tokens=False)
    end_tokens       = tokenizer.encode(end_token, add_special_tokens=False)

    for example in batch:
        messages = example["messages"]

        # ChatML 형식으로 prompt 조합
        prompt = ""
        for msg in messages:
            role    = msg["role"]
            content = msg["content"].strip()
            prompt += f"{start_token}{role}\n{content}{end_token}"

        # 토크나이징
        tokenized     = tokenizer(
            prompt,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None
        )
        input_ids     = tokenized["input_ids"]
        attention_mask= tokenized["attention_mask"]
        labels        = [-100] * len(input_ids)

        # assistant 세그먼트만 레이블에 복사 (종료 토큰 포함)
        i = 0
        n = len(input_ids)
        while i <= n - len(assistant_tokens):
            if input_ids[i:i + len(assistant_tokens)] == assistant_tokens:
                start_idx = i + len(assistant_tokens)
                end_idx   = start_idx
                # <|im_end|> 위치까지 찾기
                while end_idx <= n - len(end_tokens):
                    if input_ids[end_idx:end_idx + len(end_tokens)] == end_tokens:
                        end_idx += len(end_tokens)
                        break
                    end_idx += 1
                # start_idx 부터 end_idx 직전까지, 즉 응답 본문+종료 토큰 모두 레이블링
                for j in range(start_idx, end_idx):
                    labels[j] = input_ids[j]
                i = end_idx
            else:
                i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    # 패딩 및 Tensor 변환
    max_len = max(len(ids) for ids in new_batch["input_ids"])
    for idx in range(len(new_batch["input_ids"])):
        pad_len = max_len - len(new_batch["input_ids"][idx])
        new_batch["input_ids"][idx].extend([tokenizer.pad_token_id] * pad_len)
        new_batch["attention_mask"][idx].extend([0] * pad_len)
        new_batch["labels"][idx].extend([-100] * pad_len)

    for k in new_batch:
        new_batch[k] = torch.tensor(new_batch[k])

    return new_batch

collate_fn(batch) 함수는 자연어 처리 모델 학습을 위해 데이터를 전처리하는 역할을 수행합니다. 이 함수는 배치 내의 데이터를 처리하여 모델이 사용할 수 있는 입력 형식으로 변환합니다.됩니다.

먼저, 각 샘플의 메시지에서 개행 문자를 제거하고 필요한 정보만 남깁니다. 정리된 메시지로 텍스트를 구성하고 이를 토큰화하여 input_ids와 attention_mask를 생성합니다. 이후 레이블 데이터를 초기화한 다음 assistant 응답을 찾아 해당 범위에 레이블을 설정합니다. 이 범위를 제외한 나머지 위치는 -100으로 설정하여 손실 계산에서 제외되도록 합니다.

In [30]:
# 최대 길이
max_seq_length=8192

# collate_fn 테스트 (배치 크기 1로)
example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 2390])
어텐션 마스크 형태: torch.Size([1, 2390])
레이블 형태: torch.Size([1, 2390])


In [31]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[151644, 8948, 198, 64795, 82528, 33704, 58034, 129044, 136671, 20401, 15235, 58034, 125786, 55054, 78952, 13, 128677, 125512, 32831, 20401, 144484, 58034, 125786, 16186, 139713, 382, 81650, 31328, 23573, 40720, 131958, 132270, 3034, 25, 547, 15, 15, 17, 198, 57268, 127478, 37195, 254, 137771, 25, 220, 17, 15, 17, 19, 12, 16, 15, 12, 16, 19, 271, 2, 13852, 271, 2610, 1231, 1618, 825, 476, 803, 5746, 311, 7789, 448, 279, 1196, 3239, 382, 2610, 525, 3897, 448, 729, 32628, 2878, 366, 15918, 1472, 15918, 29, 11874, 9492, 510, 27, 15918, 397, 4913, 1313, 788, 330, 1688, 497, 330, 1688, 788, 5212, 606, 788, 330, 1836, 9840, 497, 330, 4684, 788, 330, 55902, 125678, 79632, 19391, 142976, 10764, 92120, 130109, 29346, 18411, 133970, 42905, 138017, 17877, 85322, 77226, 126204, 11, 69441, 231, 126333, 143802, 42039, 36055, 144210, 52300, 98801, 18411, 94879, 60838, 13, 126871, 29326, 90711, 112, 130229, 34395, 28002, 28019, 8, 17380, 68408, 75528, 33861, 136849, 12802, 95351, 608

In [32]:
# 디코딩된 input_ids 출력
decoded_text = tokenizer.decode(
    batch["input_ids"][0].tolist(),
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\ninput_ids 디코딩 결과:")
print(decoded_text)


input_ids 디코딩 결과:
<|im_start|>system
당신은 상준몰의 AI 상담사입니다. 성심성의껏 상담하십시오.

로그인한 사용자의 현재 ID: U002
오늘 날짜: 2024-10-14

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "search_product", "description": "상품명에 특정 키워드를 포함하는 제품을 검색하고, 평점 순으로 정렬된 결과를 반환합니다. 필요시 카테고리(category)로 추가 필터링이 가능합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 상품명 키워드 (예: '노트북')"}, "category": {"type": "string", "description": "선택적 카테고리 필터 (예: '전자기기')"}}, "required": ["keyword"], "additionalProperties": false}}}
{"type": "function", "function": {"name": "view_order_history", "description": "사용자의 전체 주문 내역을 반환합니다. 각 주문에 대해 주문 번호, 주문 일자, 총 결제 금액, 결제 상태, 배송 상태, 택배사, 운송장 번호, 배송 진행 단계, 주문에 포함된 상품명 목록을 제공합니다. 이 함수로도 최근 주문의 배송 상태를 확인할 수 있습니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "des

In [33]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

In [34]:
# -100이 아닌 부분만 골라 디코딩
label_ids = [token_id for token_id in batch["labels"][0].tolist() if token_id != -100]

decoded_labels = tokenizer.decode(
    label_ids,
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\nlabels 디코딩 결과 (-100 제외):")
print(decoded_labels)


labels 디코딩 결과 (-100 제외):
<tool_call>
{"name": "view_order_history", "arguments": {"user_id": "U002"}}
</tool_call><|im_end|>최근 주문 내역으로는 9월 10일에 주문하신 '무선 청소기'와 8월 5일에 주문하신 '블루투스 스피커'가 있습니다. 두 주문 모두 배송 완료 상태입니다. 다른 문의 사항 있으신가요?<|im_end|><tool_call>
{"name": "view_user_profile", "arguments": {"user_id": "U002"}}
</tool_call><|im_end|>고객님의 프로필 정보는 다음과 같습니다: 이름은 홍길동, 이메일은 hong@example.com, 전화번호는 010-1234-5678, 주소는 서울특별시 강남구입니다. 현재 보유하신 포인트는 1500점이고, 멤버십 등급은 Gold입니다. 또한, 10% 할인 쿠폰을 보유하고 계십니다.<|im_end|>죄송하지만, 장바구니 전체를 비우는 기능은 제공되지 않습니다. 개별 상품을 제거하시려면 상품명을 알려주시면 도와드리겠습니다.<|im_end|>알겠습니다. 더 궁금하신 사항이 있으신가요?<|im_end|>네, 문의해 주셔서 감사합니다. 상준몰 고객센터 AI 상담사였습니다. 좋은 하루 되세요!<|im_end|>


## 5. 전처리 이해하기

**input_ids와 labels는 어떻게 생성되는가?**

LLM 학습에서 `input_ids`와 `labels`는 모델의 학습 목표에 따라 생성됩니다. 시스템 프롬프트까지 포함하여 설명하겠습니다.

예를 들어, 다음과 같은 대화 데이터를 모델이 학습해야 한다고 가정합니다:  
- 시스템 프롬프트: `당신은 친절하고 도움이 되는 AI 어시스턴트입니다.`  
- 사용자 메시지: `안녕하세요, 오늘 날씨는 어떤가요?`  
- 어시스턴트 응답: `안녕하세요! 오늘 날씨는 맑고 화창합니다.`  

Qwen에서는 다음과 같은 템플릿 구조를 사용합니다(줄바꿈 포함, 단 `<|im_end|>` 앞에는 줄바꿈 없음):

```python
<|im_start|>system
당신은 친절하고 도움이 되는 AI 어시스턴트입니다.<|im_end|>
<|im_start|>user
안녕하세요, 오늘 날씨는 어떤가요?<|im_end|>
<|im_start|>assistant
안녕하세요! 오늘 날씨는 맑고 화창합니다.<|im_end|>
```

이 전체 텍스트는 토크나이저에 의해 정수 시퀀스로 변환해봅시다.  
(실제와 다르고 가정하여 정수를 맵핑하겠습니다.)

먼저 모든 특수 토큰들은 아래의 고유 ID를 가진다고 가정해봅시다.  
- `<|im_start|>` = 토큰 ID 1  
- `<|im_end|>` = 토큰 ID 2  
- 줄바꿈 = 토큰 ID 3  

역할 토큰들은 아래의 고유 ID를 가진다고 가정해봅시다.  
- system = 토큰 ID 4  
- user = 토큰 ID 5  
- assistant = 토큰 ID 6  

전체 통합된 input_ids는 다음과 같습니다:  
`input_ids = [1, 4, 3, 7, 8, 9, 2, 1, 5, 3, 10, 11, 12, 13, 14, 2, 1, 6, 3, 15, 16, 17, 18, 19, 2]`

각 부분을 분리하면:  
- 시스템 프롬프트 부분: [1, 4, 3, 7, 8, 9, 2]  
- 사용자 메시지 부분: [1, 5, 3, 10, 11, 12, 13, 14, 2]  
- 어시스턴트 응답 부분: [1, 6, 3, 15, 16, 17, 18, 19, 2]  

모델이 예측해야 할 영역은 assistant의 응답 부분인 `안녕하세요! 오늘 날씨는 맑고 화창합니다.`에 해당하는 토큰들입니다. 따라서 `labels`는 다음과 같이 설정됩니다:

`labels = [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 15, 16, 17, 18, 19, 2]`

여기서 주목할 점:  
1. 시스템 프롬프트와 사용자 메시지에 해당하는 모든 토큰은 `-100`으로 마스킹됩니다.  
2. 어시스턴트 헤더 (`<|im_start|>assistant\n`)도 `-100`으로 마스킹됩니다.  
3. 실제 어시스턴트 응답 내용(15-19)과 종료 토큰 `<|im_end|>`(2)까지 포함해서 label로 사용해야 합니다.  
4. 이렇게 해야 모델이 적절한 시점에서 종료할 수 있도록 학습됩니다.

이처럼 `labels`는 모델이 실제로 생성해야 할 출력 부분만을 포함하고, 나머지 부분은 `-100`으로 채워져 손실 계산에서 제외됩니다. 이를 통해 모델은 입력(시스템 프롬프트+사용자 질문)을 기반으로 적절한 응답을 생성하는 방법을 학습합니다.

학습 과정에서는:  
1. 모델에 `input_ids` 전체를 입력으로 제공합니다.  
2. 모델은 각 위치에서 다음 토큰을 예측합니다.  
3. 손실 계산 시 `labels`가 `-100`이 아닌 위치에서만 오차를 계산합니다.  
4. 이를 통해 모델은 주어진 맥락(시스템 프롬프트와 사용자 질문)에 대해 적절한 응답을 생성하는 방법을 학습합니다.

## 6. 학습

In [35]:
trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이 설정
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [36]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,0.801500
20,0.549200
30,0.450700
40,0.349300
50,0.337100
60,0.322200
70,0.274500
80,0.259300
90,0.230000
100,0.225300


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-pac

## 7. 테스트 데이터 준비하기

In [6]:
import re
from typing import List, Dict

In [7]:
def to_chatml(data):
    """
    data: messages 리스트이거나 {"messages": [...]} 형태의 dict
    반환값: ChatML 포맷의 문자열
    """
    # data가 dict이고 'messages' 키가 있으면 messages 리스트를 꺼내고,
    # 아니면 data 자체를 messages 리스트로 간주
    messages = data.get("messages") if isinstance(data, dict) and "messages" in data else data

    parts = []
    for msg in messages:
        role = msg["role"]
        content = msg["content"]
        parts.append(f"<|im_start|>{role}\n{content}<|im_end|>")
    return "\n".join(parts)

In [8]:
print(test_dataset[0])

{'messages': [{'content': '당신은 상준몰의 AI 상담사입니다. 성심성의껏 상담하십시오.\n\n로그인한 사용자의 현재 ID: U002\n오늘 날짜: 2023-07-16\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "view_order_details", "description": "특정 주문(order_id)의 상세 내역을 조회합니다. 주문에 포함된 각 상품의 ID, 상품명, 수량, 단가, 할인 적용 가격을 반환합니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "description": "주문 상세를 조회할 사용자의 고유 식별자 (예: \'U001\')"}, "order_id": {"type": "string", "description": "상세 내역을 조회할 주문의 고유 식별자 (예: \'O001\')"}}, "required": ["user_id", "order_id"], "additionalProperties": false}}}\n{"type": "function", "function": {"name": "show_cart", "description": "지정된 사용자(user_id)의 현재 장바구니에 담긴 모든 상품을 반환합니다. 각 항목에 대해 상품 ID, 상품명, 가격, 수량, 장바구니에 추가된 일시를 포함합니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "description": "장바구니를 조회할 사용자

In [9]:
chatml_test_dataset = to_chatml(test_dataset[0])
print(chatml_test_dataset)

<|im_start|>system
당신은 상준몰의 AI 상담사입니다. 성심성의껏 상담하십시오.

로그인한 사용자의 현재 ID: U002
오늘 날짜: 2023-07-16

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "view_order_details", "description": "특정 주문(order_id)의 상세 내역을 조회합니다. 주문에 포함된 각 상품의 ID, 상품명, 수량, 단가, 할인 적용 가격을 반환합니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "description": "주문 상세를 조회할 사용자의 고유 식별자 (예: 'U001')"}, "order_id": {"type": "string", "description": "상세 내역을 조회할 주문의 고유 식별자 (예: 'O001')"}}, "required": ["user_id", "order_id"], "additionalProperties": false}}}
{"type": "function", "function": {"name": "show_cart", "description": "지정된 사용자(user_id)의 현재 장바구니에 담긴 모든 상품을 반환합니다. 각 항목에 대해 상품 ID, 상품명, 가격, 수량, 장바구니에 추가된 일시를 포함합니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "description": "장바구니를 조회할 사용자의 고유 식별자 (예: 'U001')"}},

In [10]:
def extract_examples(chatml: str) -> List[Dict[str, str]]:
    """
    ChatML 문자열에서 각 assistant 응답을 분리하여
    'input'과 'label' 쌍을 생성합니다.
    'input'은 해당 assistant 응답 직전까지의 모든 대화 + '<|im_start|>assistant',
    'label'은 해당 assistant의 응답 내용입니다.
    """
    examples: List[Dict[str, str]] = []
    pattern = re.compile(r'<\|im_start\|>assistant(.*?)(?=<\|im_end\|>)', re.DOTALL)

    for match in pattern.finditer(chatml):
        start_idx = match.start()
        input_text = chatml[:start_idx].strip() + '\n<|im_start|>assistant'
        label_text = match.group(1).strip()
        examples.append({
            "input": input_text,
            "label": label_text
        })

    return examples

**`extract_examples` 함수 개요**  
ChatML 포맷의 대화 문자열에서 `<|im_start|>assistant` 응답 부분을 찾아 그 이전까지의 대화를 `input`으로 응답 내용을 `label`로 추출합니다.

이 작업은 대화 중 각 assistant 응답을 하나씩 분리하여 평가할 수 있도록 전처리하는 과정입니다.  
따라서 assistant의 각 응답을 하나의 평가 단위로 분리하여, **“이전까지의 흐름에 비추어 이 응답이 적절했는가”**를 다양한 시나리오에 대해서 확인할 수 있도록 구성합니다.

---

### 1) 예시 ChatML 입력
```python
<|im_start|>system
시스템 메시지입니다.<|im_end|>
<|im_start|>user
안녕하세요<|im_end|>
<|im_start|>assistant
안녕하세요. 무엇을 도와드릴까요?<|im_end|>
<|im_start|>user
상품을 추천해주세요.<|im_end|>
<|im_start|>assistant
어떤 카테고리 상품을 찾으시나요?<|im_end|>
```
---

### 2) 추출 결과 예시
--
```python
[
  {
    "input": "<|im_start|>system\n시스템 메시지입니다.<|im_end|>\n<|im_start|>user\n안녕하세요<|im_end|>\n<|im_start|>assistant",
    "label": "안녕하세요. 무엇을 도와드릴까요?"
  },
  {
    "input": "<|im_start|>system\n시스템 메시지입니다.<|im_end|>\n<|im_start|>user\n안녕하세요<|im_end|>\n<|im_start|>assistant\n안녕하세요. 무엇을 도와드릴까요?<|im_end|>\n<|im_start|>user\n상품을 추천해주세요.<|im_end|>\n<|im_start|>assistant",
    "label": "어떤 카테고리 상품을 찾으시나요?"
  }
]
```

- 첫 번째 예제의 `input`에는 첫 assistant 응답 이전의 system·user 메시지가 포함되고,  
  `label`에는 첫 assistant 응답만 담깁니다.  
- 두 번째 예제는 두 번째 assistant 응답 이전의 모든 대화를 `input`으로,  
  “어떤 카테고리 상품을 찾으시나요?”를 `label`로 추출합니다.

In [11]:
test_examples = extract_examples(chatml_test_dataset)

In [12]:
# 전체 예제 수 확인
print('전체 예제 수 확인:', len(test_examples))
print('--' * 50)
# 첫 번째 예제의 input과 label 출력
print('첫번째 입력:', test_examples[0]['input'])
print('--' * 50)
print('첫번째 레이블:', test_examples[0]['label'])
print('--' * 50)
print('두번째 입력:', test_examples[1]['input'])
print('--' * 50)
print('두번째 레이블:', test_examples[1]['label'])
print('--' * 50)

전체 예제 수 확인: 8
----------------------------------------------------------------------------------------------------
첫번째 입력: <|im_start|>system
당신은 상준몰의 AI 상담사입니다. 성심성의껏 상담하십시오.

로그인한 사용자의 현재 ID: U002
오늘 날짜: 2023-07-16

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "view_order_details", "description": "특정 주문(order_id)의 상세 내역을 조회합니다. 주문에 포함된 각 상품의 ID, 상품명, 수량, 단가, 할인 적용 가격을 반환합니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "description": "주문 상세를 조회할 사용자의 고유 식별자 (예: 'U001')"}, "order_id": {"type": "string", "description": "상세 내역을 조회할 주문의 고유 식별자 (예: 'O001')"}}, "required": ["user_id", "order_id"], "additionalProperties": false}}}
{"type": "function", "function": {"name": "show_cart", "description": "지정된 사용자(user_id)의 현재 장바구니에 담긴 모든 상품을 반환합니다. 각 항목에 대해 상품 ID, 상품명, 가격, 수량, 장바구니에 추가된 일시를 포함합니다.", "paramet

In [13]:
prompt_lst = []
label_lst = []

for item in test_dataset:
    chatml = to_chatml(item)  # ChatML 문자열로 변환
    examples = extract_examples(chatml)  # assistant 응답 단위로 분리

    for ex in examples:
        prompt_lst.append(ex['input'])
        label_lst.append(ex['label'])

In [14]:
print('기존 테스트 데이터의 개수:', len(test_dataset))
print('턴 별 분리 후 테스트 데이터의 개수:', len(prompt_lst))

기존 테스트 데이터의 개수: 77
턴 별 분리 후 테스트 데이터의 개수: 655


In [15]:
print(prompt_lst[10])

<|im_start|>system
당신은 상준몰의 AI 상담사입니다. 성심성의껏 상담하십시오.

로그인한 사용자의 현재 ID: U006
오늘 날짜: 2024-02-02

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "add_to_cart", "description": "사용자의 장바구니에 지정된 상품(product_id)과 수량(quantity)을 추가합니다. 동일 상품이 이미 있으면 수량을 증가시키고, 새 항목으로 추가합니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "description": "장바구니에 상품을 추가할 사용자의 고유 식별자 (예: 'U001')"}, "product_id": {"type": "string", "description": "장바구니에 추가할 상품의 고유 식별자 (예: 'P003')"}, "quantity": {"type": "integer", "description": "추가할 상품 수량 (기본값: 1)", "default": 1, "minimum": 1}}, "required": ["user_id", "product_id"], "additionalProperties": false}}}
{"type": "function", "function": {"name": "view_order_history", "description": "사용자의 전체 주문 내역을 반환합니다. 각 주문에 대해 주문 번호, 주문 일자, 총 결제 금액, 결제 상태, 배송 상태, 택배사, 운송장 번호, 배송 진행 단계, 주문에 포함된 상품명 목록을 제공

In [16]:
print(label_lst[10])

<tool_call>
{"name": "search_product", "arguments": {"keyword": "노트북"}}
</tool_call>


## 8. 파인튜닝 모델 테스트

`AutoPeftModelForCausalLM()`의 입력으로 LoRA Adapter가 저장된 체크포인트의 주소를 넣으면 LoRA Adapter가 기존의 LLM과 부착되어 로드됩니다. 이 과정은 LoRA Adapter의 가중치를 사전 학습된 언어 모델(LLM)에 통합하여 미세 조정된 모델을 완성하는 것을 의미합니다.

`peft_model_id` 변수는 미세 조정된 가중치가 저장된 체크포인트의 경로를 나타냅니다. `"llama3-8b-summarizer-ko/checkpoint-372"`는 LoRA Adapter 가중치가 저장된 위치로, 이 경로에서 해당 가중치를 불러옵니다.

`fine_tuned_model`은 `AutoPeftModelForCausalLM.from_pretrained` 메서드를 통해 체크포인트를 로드하여 생성됩니다. 이 메서드는 LLM과 LoRA Adapter를 결합하고, 최적화된 설정으로 모델을 메모리에 로드합니다. `device_map="auto"` 옵션은 모델을 자동으로 GPU에 배치합니다.

`pipeline`은 Hugging Face의 고수준 유틸리티로, NLP 작업(예: 텍스트 생성, 번역, 요약 등)을 간단히 수행할 수 있게 해줍니다. 이 코드에서 사용된 `pipeline("text-generation")`은 텍스트 생성 작업을 수행하기 위한 파이프라인 객체를 생성합니다. 파이프라인은 내부적으로 모델과 토크나이저를 관리하여, 입력 텍스트를 토큰화하고, 모델을 통해 생성된 결과를 다시 디코딩하여 사람이 읽을 수 있는 텍스트로 변환합니다.

이 코드는 미세 조정된 LLM을 로드한 뒤, 이를 이용해 텍스트 생성 작업을 간단히 수행할 수 있도록 준비하는 데 목적이 있습니다. `pipeline`을 통해 텍스트 생성 작업을 실행하면, 입력 텍스트에 기반하여 모델이 다음 토큰을 예측하고 이를 반복적으로 생성합니다. 이 과정은 사용자에게 자연스러운 텍스트를 출력하는 데 사용됩니다.데 사용됩니다.

In [18]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [19]:
# 마지막 학습 모델 로드
peft_model_id = "qwen-2.5-7b-function-calling/checkpoint-234"

fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [20]:
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

In [21]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

### 파인 튜닝 모델 호출

In [22]:
for prompt, label in zip(prompt_lst[70:75], label_lst[70:75]):
    print(f"    user:\n{prompt}")
    print('--' * 10)
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print('--' * 10)
    print(f"    label:\n{label}")
    print("=="*50)

    user:
<|im_start|>system
당신은 상준몰의 AI 상담사입니다. 성심성의껏 상담하십시오.

로그인한 사용자의 현재 ID: U010
오늘 날짜: 2025-11-30

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "search_product", "description": "상품명에 특정 키워드를 포함하는 제품을 검색하고, 평점 순으로 정렬된 결과를 반환합니다. 필요시 카테고리(category)로 추가 필터링이 가능합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 상품명 키워드 (예: '노트북')"}, "category": {"type": "string", "description": "선택적 카테고리 필터 (예: '전자기기')"}}, "required": ["keyword"], "additionalProperties": false}}}
{"type": "function", "function": {"name": "view_user_profile", "description": "사용자의 기본 프로필 정보(이름, 이메일, 전화번호, 주소, 포인트, 멤버십)와 보유 쿠폰 정보를 반환합니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "description": "프로필을 조회할 사용자의 고유 식별자, 예: 'U001'"}}, "required": ["user_id"], "additionalProperties"

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train 

    response:
장바구니에서 '우산' 관련 상품이 성공적으로 제거되었습니다.
--------------------
    label:
장바구니에서 '우산' 관련 상품이 성공적으로 제거되었습니다.
    user:
<|im_start|>system
당신은 상준몰의 AI 상담사입니다. 성심성의껏 상담하십시오.

로그인한 사용자의 현재 ID: U010
오늘 날짜: 2025-11-30

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "search_product", "description": "상품명에 특정 키워드를 포함하는 제품을 검색하고, 평점 순으로 정렬된 결과를 반환합니다. 필요시 카테고리(category)로 추가 필터링이 가능합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 상품명 키워드 (예: '노트북')"}, "category": {"type": "string", "description": "선택적 카테고리 필터 (예: '전자기기')"}}, "required": ["keyword"], "additionalProperties": false}}}
{"type": "function", "function": {"name": "view_user_profile", "description": "사용자의 기본 프로필 정보(이름, 이메일, 전화번호, 주소, 포인트, 멤버십)와 보유 쿠폰 정보를 반환합니다.", "parameters": {"type": "object", "properties": {"user_id": {"ty

## 9. 기본 모델 테스트

이번에는 LoRA Adapter를 merge하지 않은 기본 모델로 테스트 데이터에 대해서 인퍼런스해보겠습니다.


In [25]:
base_model_id = "Qwen/Qwen2.5-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### 기본 모델 호출

In [26]:
for prompt, label in zip(prompt_lst[70:75], label_lst[70:75]):
    print(f"    user:\n{prompt}")
    print('--' * 10)
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print('--' * 10)
    print(f"    label:\n{label}")
    print("=="*50)

    user:
<|im_start|>system
당신은 상준몰의 AI 상담사입니다. 성심성의껏 상담하십시오.

로그인한 사용자의 현재 ID: U010
오늘 날짜: 2025-11-30

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "search_product", "description": "상품명에 특정 키워드를 포함하는 제품을 검색하고, 평점 순으로 정렬된 결과를 반환합니다. 필요시 카테고리(category)로 추가 필터링이 가능합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 상품명 키워드 (예: '노트북')"}, "category": {"type": "string", "description": "선택적 카테고리 필터 (예: '전자기기')"}}, "required": ["keyword"], "additionalProperties": false}}}
{"type": "function", "function": {"name": "view_user_profile", "description": "사용자의 기본 프로필 정보(이름, 이메일, 전화번호, 주소, 포인트, 멤버십)와 보유 쿠폰 정보를 반환합니다.", "parameters": {"type": "object", "properties": {"user_id": {"type": "string", "description": "프로필을 조회할 사용자의 고유 식별자, 예: 'U001'"}}, "required": ["user_id"], "additionalProperties"